In [26]:
import numpy as np
import pandas as pd

In [27]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("sartajbhuvaji/brain-tumor-classification-mri")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/sartajbhuvaji/brain-tumor-classification-mri/versions/2


In [28]:
import os
import cv2
import numpy as np

# Define dataset path
train_dir = os.path.join(path, 'Training')

# Use the correct folder names from os.listdir
labels = ['pituitary_tumor', 'glioma_tumor', 'meningioma_tumor', 'no_tumor']

X_train = []
y_train = []
image_size = 150

for i in labels:
    folderPath = os.path.join(train_dir, i)

    # Check if the directory exists
    if not os.path.exists(folderPath):
        print(f"Warning: {folderPath} does not exist! Skipping...")
        continue

    for j in os.listdir(folderPath):
        img_path = os.path.join(folderPath, j)

        # Process only image files
        if j.endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (image_size, image_size))
            X_train.append(img)
            y_train.append(labels.index(i))  # Store label as integer

In [29]:
test_dir = os.path.join(path, 'Testing')
for i in labels:
    folderPath = os.path.join(train_dir, i)

    # Check if the directory exists
    if not os.path.exists(folderPath):
        print(f"Warning: {folderPath} does not exist! Skipping...")
        continue

    for j in os.listdir(folderPath):
        img_path = os.path.join(folderPath, j)

        # Process only image files
        if j.endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (image_size, image_size))
            X_train.append(img)
            y_train.append(labels.index(i))  # Store label as integer

In [30]:
X_train = np.array(X_train, dtype=np.float32) / 255.0  # Normalize pixel values
y_train = np.array(y_train, dtype=np.int32)

In [31]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=101)
X_train.shape

(5740, 150, 150, 3)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=101)

In [33]:
import tensorflow as tf
from tensorflow import keras
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [34]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Initialize the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

# Flatten before feeding into dense layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))  # 4 classes

# Model Summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 146, 146, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 73, 73, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 73, 73, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 71, 71, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 69, 69, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 34, 34, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 34, 34, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 30, 30, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 28, 28, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 12, 12, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 10, 10, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 5, 5, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 5, 5, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │       3,277,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 4,447,044 (16.96 MB)

 Trainable params: 4,447,044 (16.96 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [36]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1)

Epoch 1/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 25s 120ms/step - accuracy: 0.2853 - loss: 1.3670 - val_accuracy: 0.3065 - val_loss: 1.3334
Epoch 2/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.2929 - loss: 1.3532 - val_accuracy: 0.4217 - val_loss: 1.2984
Epoch 3/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.3736 - loss: 1.2529 - val_accuracy: 0.5761 - val_loss: 0.9663
Epoch 4/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.6021 - loss: 0.9355 - val_accuracy: 0.7587 - val_loss: 0.7066
Epoch 5/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - accuracy: 0.7285 - loss: 0.6494 - val_accuracy: 0.8174 - val_loss: 0.4663
Epoch 6/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - accuracy: 0.7982 - loss: 0.4886 - val_accuracy: 0.8717 - val_loss: 0.3397
Epoch 7/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.8691 - loss: 0.3421 - val_accuracy: 0.9065 - val_loss: 0.2518
Epoch 8/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.9055 - loss: 0.2588 - val

In [37]:
history.history

{'accuracy': [0.28484994173049927,
  0.2991287410259247,
  0.4075508117675781,
  0.653920590877533,
  0.7429816126823425,
  0.8163117170333862,
  0.8775411248207092,
  0.9140852093696594,
  0.9346563220024109,
  0.9416747093200684,
  0.9554694890975952,
  0.9537754058837891,
  0.9593417048454285,
  0.9736205339431763,
  0.9721684455871582,
  0.9721684455871582,
  0.9789448380470276,
  0.9794288277626038,
  0.9680542349815369,
  0.9784607887268066],
 'loss': [1.3575379848480225,
  1.3462533950805664,
  1.2065753936767578,
  0.8192740082740784,
  0.6228430867195129,
  0.4431939423084259,
  0.3266483545303345,
  0.2431989312171936,
  0.1842719465494156,
  0.16366666555404663,
  0.125776007771492,
  0.13247288763523102,
  0.11560416221618652,
  0.07722826302051544,
  0.07792196422815323,
  0.08775608986616135,
  0.06666462123394012,
  0.06229035183787346,
  0.09719253331422806,
  0.06388169527053833],
 'val_accuracy': [0.3065217435359955,
  0.4217391312122345,
  0.5760869383811951,
  0.758

In [38]:
model.evaluate(X_test,y_test)

36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.9725 - loss: 0.0668


[0.07138348370790482, 0.9799651503562927]

In [39]:
model.save('Brain_Tumor_Detection.h5')

In [41]:
from google.colab import files
files.download('Brain_Tumor_Detection.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>